# Mathematical Program MultibodyPlant Tutorial
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


다음과 같은 예제를 살펴보자.:
* IIWA arm을 가지는 `MultibodyPlant` 생성하기
* `float`과 `AutoDiffXd` 입력을 처리할 수 있는 `MathematicalProgram`에 대한 커스텀 평가자(evaluator)를 작성해서 간단한 IK 문제 해결하기
* 제약에 대한 커스텀 평가자 사용하기
* 비용에 대한 커스텀 평가자 사용하기

***추가될 내용***:
* `pydrake.multibody.inverse_kinematics` 사용하기
* Meshcat으로 시각화하기

### Important Note

커스텀 평가자를 작성하기 전에 `MultibodyPlant`와 함께 사용할 IK 문제를 효과적으로 해결하기 위해서는 [API for `pydrake.multibody.inverse_kinematics`](
https://drake.mit.edu/pydrake/pydrake.multibody.inverse_kinematics.html)를 먼저 검토하는 것이 좋다. 여러분이 원하는 기능을 거기서 찾을 수 있을지도 모른다.

## Inverse Kinematics Problem

이 튜토리얼에서는 간단한 IK문제를 풀어 링크 7의 원점을 목표 위치로부터 일정 거리에 둘 것이다. `MathematicalProgram`을 사용하여 두 가지 방법으로 이 문제를 해결할 것이다. 첫 번째 방법은 평가자를 제약 조건(최소 및 최대 거리)으로 사용하고, 두 번째 방법은 평가자를 비용(가능한 근접)으로 사용하는 것이다.

`MathematicalProgram`에 대한 자세한 내용은 [`MathematicalProgram` Tutorial](./mathematical_program.ipynb)을 참조하자.

## Setup

먼저, 필요한 모듈을 불러오고 IIWA를 포함하는 `MultibodyPlant`를 로딩한다.

In [ ]:
import numpy as np

from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.systems.analysis import Simulator
from pydrake.all import MultibodyPlant

from pydrake.solvers import MathematicalProgram, Solve

In [ ]:
plant_f = MultibodyPlant(0.0)
iiwa_url = (
   "package://drake/manipulation/models/iiwa_description/sdf/"
   "iiwa14_no_collision.sdf")
(iiwa,) = Parser(plant_f).AddModels(url=iiwa_url)

# Define some short aliases for frames.
W = plant_f.world_frame()
L0 = plant_f.GetFrameByName("iiwa_link_0", iiwa)
L7 = plant_f.GetFrameByName("iiwa_link_7", iiwa)

plant_f.WeldFrames(W, L0)
plant_f.Finalize()

## Writing our Custom Evaluator

우리의 평가자는 `link_7_distance_to_target`을 사용하여 구현되었다. 왜냐하면 `inverse_kinematics` 서브모듈에 이미 존재하는 클래스에서 해당 기능을 제공하지 않기 때문이다.

Python으로 커스텀 평가자를 작성하려면 `link_7_distance_to_target`의 구현에서 확인할 수 있듯이 `float` 및 `AutoDiffXd` 입력을 명확히 확인해야만 한다.

In [ ]:
# Allocate float context to be used by evaluators.
context_f = plant_f.CreateDefaultContext()
# Create AutoDiffXd plant and corresponding context.
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

def resolve_frame(plant, F):
    """Gets a frame from a plant whose scalar type may be different."""
    return plant.GetFrameByName(F.name(), F.model_instance())

# Define target position.
p_WT = [0.1, 0.1, 0.6]

def link_7_distance_to_target(q):
    """Evaluates squared distance between L7 origin and target T."""
    # Choose plant and context based on dtype.
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        # Assume AutoDiff.
        plant = plant_ad
        context = context_ad
    # Do forward kinematics.
    plant.SetPositions(context, iiwa, q)
    X_WL7 = plant.CalcRelativeTransform(
        context, resolve_frame(plant, W), resolve_frame(plant, L7))
    p_TL7 = X_WL7.translation() - p_WT
    return p_TL7.dot(p_TL7)

# WARNING: If you return a scalar for a constraint, or a vector for
# a cost, you may get the following cryptic error:
# "Unable to cast Python instance to C++ type"
link_7_distance_to_target_vector = lambda q: [link_7_distance_to_target(q)]

## 최적화 문제에 대한 공식 만들기(Formulating the Optimization Problems)

### Formluation 1: Using the Custom Evaluator in a Constraint

기본 비용과 제약 조건으로 커스텀 평가자를 사용하여 문제에 대한 공식을 만들고 해결할 것이다.

평가자의 벡터화된 버전을 사용하는 점에 유의하자.

In [ ]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# Define nominal configuration.
q0 = np.zeros(plant_f.num_positions())

# Add basic cost. (This will be parsed into a QuadraticCost.)
prog.AddCost((q - q0).dot(q - q0))

# Add constraint based on custom evaluator.
prog.AddConstraint(
    link_7_distance_to_target_vector,
    lb=[0.1], ub=[0.2], vars=q)

In [ ]:
result = Solve(prog, initial_guess=q0)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")

### Formulation 2: Using Custom Evaluator in a Cost

이 문제에 대한 공식을 만들고 해결한다. 하지만 이번에는 비용에 대한 커스텀 평가자를 사용한다.

평가자의 scalar 버전을 사용한다는 점에 유의하자.

In [ ]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# Define nominal configuration.
q0 = np.zeros(plant_f.num_positions())

# Add custom cost.
prog.AddCost(link_7_distance_to_target, vars=q)

In [ ]:
result = Solve(prog, initial_guess=q0)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")